# Control testing using pure persuit and PID controller

In [94]:
import sys
import os
import glob
import math 
import random 
import time 
import numpy as np
import cv2
import open3d as o3d
from matplotlib import cm
import carla

#  Trying to import the Global and Local 
sys.path.append(("C:/CARLA/WindowsNoEditor/PythonAPI/carla"))
from agents.navigation.global_route_planner import GlobalRoutePlanner

# To import a basic agent
from agents.navigation.basic_agent import BasicAgent

# To import a behavior agent
from agents.navigation.behavior_agent import BehaviorAgent


In [95]:
'''' Trying something'''

class PIDController:
    def __init__(self, Kp, Ki, Kd, set_point):
        self.Kp = Kp
        self.Ki = Ki
        self.Kd = Kd
        self.set_point = set_point
        self.int_term = 0
        self.derivative_term = 0
        self.last_error = None

    def get_control(self, measurement, dt):
        error = self.set_point - measurement
        self.int_term += error*self.Ki*dt
        if self.last_error is not None:
            self.derivative_term = (error-self.last_error)/dt*self.Kd
        self.last_error = error
        return self.Kp * error + self.int_term + self.derivative_term
    


In [96]:
''' Connecting to Carla and fetching blue-print of the map'''

# Connect the client and set up bp library and spawn point
client = carla.Client('localhost', 2000)
client.set_timeout(20.0)
world = client.get_world()
bp_lib = world.get_blueprint_library() 
spawn_points = world.get_map().get_spawn_points() 

In [97]:
start_location = random.choice(spawn_points)
end_location = random.choice(spawn_points)
print(end_location)
start_point = start_location.location
end_point = end_location.location

Transform(Location(x=-48.839951, y=-32.034920, z=0.600000), Rotation(pitch=0.000000, yaw=90.432327, roll=0.000000))


In [99]:
Kdd = 0.4
FPS = 30
dt  = 1./FPS
L = 2.875 #### Will need to check how to get 
param_Kp = 0
param_Ki = 0
param_Kd = 0
desired_vel = 25
pid=PIDController(param_Kp, param_Ki, param_Kd, 0)

In [100]:
def get_target_wp_index(veh_location, waypoint_list):
    print(f'the last location in the list {waypoint_list[-1]}')
    info = False
    exception = False
    dxl, dyl = [], []
    for i in range(len(waypoint_list)):
        dx = abs(veh_location.x - waypoint_list[i][0])
        dxl.append(dx) #list of dx1
        dy = abs(veh_location.y - waypoint_list[i][1])
        dyl.append(dy)

    dist = np.hypot(dxl, dyl)
    try:
        idx = np.argmin(dist) + 4 
    except: idx = 4
    # take closest waypoint, else last wp

    if idx < len(waypoint_list):
        print(f'This line is printed')
        tx = waypoint_list[idx][0]
        ty = waypoint_list[idx][1]

    else:
        print(f'the exception is printed')
        tx = waypoint_list[-1][0]
        ty = waypoint_list[-1][1]
        exception = True
    print(f'The current waypoint is {(tx, ty)}')

    if (tx, ty) == waypoint_list[-1] and exception == False:
        info = True
    return idx, (tx, ty), info

In [101]:
'''Tracing a route between the two specified point '''

sampling_resolution = 1
grp = GlobalRoutePlanner(world.get_map(), sampling_resolution)
route =  grp.trace_route(start_point, end_point)

''' This is just to visualize the route'''
for waypoint in route:
    world.debug.draw_string(waypoint[0].transform.location, '^', draw_shadow=False,
        color=carla.Color(r=0, g=0, b=255), life_time=60.0,
        persistent_lines=True)
    
print("Done")

Done


In [102]:
waypoint_lst = []
i = 0
for waypoint, _ in route:
    i+=1
    loc = waypoint.transform.location
    waypoint_lst.append((loc.x,loc.y)) 
    
print(len(waypoint_lst),i)
    # print(asadbhai)

160 160


In [103]:
def process_img(image):
    i = np.array(image.raw_data)
    i2 = i.reshape((IM_HEIGHT, IM_WIDTH, 4))
    i3 = i2[:, :, :3]
    cv2.imshow("", i3)
    cv2.waitKey(1)
    return i3/255.0

'''SPAWNING'''
actor_lst = list()


'''spawning vehicle'''
v_bp = bp_lib.filter("vehicle.*")
# for v in v_bp:
#     print(v.id)
# print(v_bp)

v_bp = bp_lib.filter('vehicle.tesla.model3')[0]

vehicle = world.try_spawn_actor(v_bp,start_location)



'''Adding a sensor '''
IM_WIDTH = 640
IM_HEIGHT = 480
camera_bp = bp_lib.find("sensor.camera.rgb")
camera_bp.set_attribute('image_size_x', f'{IM_WIDTH}')
camera_bp.set_attribute('image_size_y', f'{IM_HEIGHT}')
camera_bp.set_attribute('fov', '110')


 # Adjust sensor relative to vehicle
camera_spawn = carla.Transform(carla.Location(x=-4, z=2.5))

# spawn the sensor and attach to vehicle.
sensor = world.try_spawn_actor(camera_bp, camera_spawn, attach_to=vehicle)

# add sensor to list of actors
actor_lst.append(sensor)

# do something with this sensor
sensor.listen(lambda data: process_img(data))


In [104]:
'''spectator mode'''


transform = carla.Transform()
transform.rotation.yaw = 180
transform.rotation.pitch = 0
transform.rotation.roll = 0

spectator = world.get_spectator() 
transform = carla.Transform(vehicle.get_transform().transform(carla.Location(x=-5,z=20)), carla.Rotation(yaw=90, pitch=-90)) 
spectator.set_transform(transform)


In [105]:
''' Creating a pure persuit controller
The target here would be the rear vehicle
'''


##### Start Your Engines #########
control = carla.VehicleControl()
control.throttle = 0.5
vehicle.apply_control(control)


pid = PIDController(param_Kp, param_Ki, param_Kd, 0)
pid.set_point = desired_vel
from tqdm import tqdm
print(f"The way point list lenghth is {len(waypoint_lst)}")
for t in range(len(waypoint_lst)):
    print(f't = {t}')
    # throttle, steer = controller.get_control(traj, speed, desired_speed=25, dt=1./FPS)

    ###### Geting Current information #########
    vehicle_transform = vehicle.get_transform()
    loc = vehicle_transform.location
    x_,y_,z_ = (loc.x,loc.y,loc.z)
    vel_array = vehicle.get_velocity() 
    x_dot, y_dot, z_dot = (vel_array.x, vel_array.y, vel_array.z)
    vel = np.hypot(x_dot, y_dot)

    print(f"The velocity = {vel}")

    yaw = vehicle_transform.rotation.yaw
    yaw = np.radians(yaw)  ### Converting in radians


    _, target, is_end = get_target_wp_index(loc, waypoint_lst)
    if is_end:
        print("You have reached your destination") 
        break
    # target = waypoint_lst[t]
    tx, ty = target
    
    
    throttle = pid.get_control(vel,dt)
    

    ######### Look ahead distance ##########
    ld_approx = np.clip(Kdd * vel ,3, 30) #### Might change in the future
    
    ######### Calculate Alpha #########3
    alpha = math.atan2((ty - y_), (tx-x_)) - yaw
    print(f"Alpha = {alpha}")

    ############### Steering Angle ####################3
    del_ = np.clip(math.atan2(2*L*math.sin(alpha),ld_approx ),-1,1)
    print(f'Steering angle = {del_}')

    control.steer = del_
    vehicle.apply_control(control)


    print('-----------------------')
    
    time.sleep(0.5)
    
control.throttle = 0
vehicle.apply_control(control)
    

The way point list lenghth is 160
t = 0
The velocity = 0.0
the last location in the list (-48.71815490722656, -33.35417556762695)
This line is printed
The current waypoint is (-82.03413391113281, 12.956045150756836)
Alpha = -2.0871159289725938e-07
Steering angle = -4.0003055305305625e-07
-----------------------
t = 1
The velocity = 1.0831633189020636e-07
the last location in the list (-48.71815490722656, -33.35417556762695)
This line is printed
The current waypoint is (-82.03413391113281, 12.956045150756836)
Alpha = -2.0871159289725938e-07
Steering angle = -4.0003055305305625e-07
-----------------------
t = 2
The velocity = 2.080447098488292e-07
the last location in the list (-48.71815490722656, -33.35417556762695)
This line is printed
The current waypoint is (-82.03413391113281, 12.956045150756836)
Alpha = -2.0871159289725938e-07
Steering angle = -4.0003055305305625e-07
-----------------------
t = 3
The velocity = 6.716281154229947e-08
the last location in the list (-48.71815490722656

In [106]:
# vehicle.destroy()
''' Destroy everything'''
try:
    car_lst = world.get_actors().filter('vehicle.*')
    print(car_lst)
    if car_lst:
        for car in car_lst:
            car.destroy()
except:
    print("Nope doesnt work")

[Actor(id=43, type=vehicle.tesla.model3)]


In [107]:
''' Lets see if I can create a arbitrary route between two points but before that lets see how carla generates a path '''

sampling_resolution = 1
grp = GlobalRoutePlanner(world.get_map(), sampling_resolution)
route =  grp.trace_route(start_point, end_point)
print(route[10][0].transform.location)
i = 0
s, y=route[i]
print((route[i]))
for x,asadbhai in route:
    loc = x.transform.location
    x_,y_,z_ = (loc.x,loc.y,loc.z) 
    
    print(vel)
    # print(asadbhai)

Location(x=-88.644112, y=12.937677, z=0.000000)
(<carla.libcarla.Waypoint object at 0x0000010B340CB690>, <RoadOption.LANEFOLLOW: 4>)
8.052826738616153
8.052826738616153
8.052826738616153
8.052826738616153
8.052826738616153
8.052826738616153
8.052826738616153
8.052826738616153
8.052826738616153
8.052826738616153
8.052826738616153
8.052826738616153
8.052826738616153
8.052826738616153
8.052826738616153
8.052826738616153
8.052826738616153
8.052826738616153
8.052826738616153
8.052826738616153
8.052826738616153
8.052826738616153
8.052826738616153
8.052826738616153
8.052826738616153
8.052826738616153
8.052826738616153
8.052826738616153
8.052826738616153
8.052826738616153
8.052826738616153
8.052826738616153
8.052826738616153
8.052826738616153
8.052826738616153
8.052826738616153
8.052826738616153
8.052826738616153
8.052826738616153
8.052826738616153
8.052826738616153
8.052826738616153
8.052826738616153
8.052826738616153
8.052826738616153
8.052826738616153
8.052826738616153
8.052826738616153
8.0

: 